## SensibleHeat Storage Volume Analysis

The basic and simplified model used to estimate the tank size required for different parts of the United States is built off of a dataset provided by OpenEI. 

[OpenEI](https://openei.org/doe-opendata/dataset/commercial-and-residential-hourly-load-profiles-for-all-tmy3-locations-in-the-united-states) created a detailed (hourly) load profile for TMY3 locations in the United States.

There is data for both residential and commercial building types, but here we only look at residential, which is broken into two load categories, HIGH and LOW. The United States is broken up into five Climate Zones, each of which is given a different type of building for decent assumptions to be made. Assumptions are broken down below: 

### Climate Zones
![](./media/climateZ.png)
### High Load Characteristics
![](./media/highLoad.png)
### Low Load Characteristics
![](./media/lowLoad.png)

### Data
For the sake of simplicity, I've cleaned up and brought over heating and domestic hot water data from a handful of TMY3 Loactions, representative of the USA. 

'Cleaning up data' consisted of summing loads on a 24 hour cylce and retrieving the max. Units are a mess. Apologies.

You may manipulate the script however you'd like, but there isn't much here... Tweak the parameters and re-run the script to see how storage requirements change.

In [1]:
# break down above for very cold/cold (low load) home and do a 
# 'BOILER REPLACEMENT GUIDE' worksheet provided by kevinsimon



## WHAT I WANT

Look at three conditions : 

EXISTING CALCULATION (to explain why it was wrong) 

12 hour load shifting - for PV shifting 

4 hour load shifting - for utility market

Can start doing all this using the stupid T_cutoff function that I made before. 



At some point, do COMMERCIAL shifting based on OpenEI dataset 



In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from array import *
import plotly.graph_objects as go
import plotly.express as px


from glob import glob

In [45]:
# Parameters For Adjustment
storageTemperature = 65 #[C]
roomTemperature = 21 #[]

standbyLosses = 5 # [%]
# shiftTime = 12 #[hrs] duration of shift that the battery is designed for

kh = 0.9 # hydronic coefficient
usableTemp = 15

# Global Variables
c_p = 4.2 #[kJ/kg]
storageLimit = 24

beta = 210e-6


hydronicEfficiency = .8 #[%]

#### Details on Adjustable Parameters

- storageTemp
    - maximum temperature you can store water at in tanks
- roomTemp
    - temperature the thermostat is set to, used to determine tCutOut

- standbyLosses
    - percent of energy lost from tanks over course of 24 hours. Depends heavily on the 
- kh
    - hydronic coefficient in kW / K. (how do I make this a value that has some useful reference frame) 
    

In [51]:
# FUNCTIONS. RUN EARLY. 
def create_df(finishString, shiftTime):
    
    
    
    update = []
    directory = os.path.join("c:\\", "/Users/hansvonclemm/Downloads/OL_data_for_watts/data")
    for root,dirs,files in os.walk(directory):
        for file in files:
            if file.endswith(finishString):
                #print (file)

                li = file.split('_')
                city = li[2].split('.')[0]
                tmy3 = int(li[2].split('.')[-1])
                path = "./data/" + file
                current = pd.read_csv(path)
                cleanDF = clean_datetime(current)
                heatNeed = get_gas_use(cleanDF, shiftTime)
                peakLoad = cleanDF['Gas:Facility [kW](Hourly)'].max()
                
                volume = get_volume_needed(heatNeed, peakLoad)
                
                shiftPercent = get_shift_percentage(cleanDF, heatNeed, storageLimit)
                
                expansion = calc_expansion(volume)

                update.append([tmy3, city, heatNeed, peakLoad, (volume*264), shiftPercent, (expansion*264)])
                
    return pd.DataFrame(update, columns=['tmy3', 'city', 'heatNeed','peakLoad', 'volume', 'shiftPercent', 'expansion'])


def clean_datetime(df):
    pat = '(?P<month>\d{2})/(?P<day>\d{2})  (?P<hour>\d{2}):(?P<minute>\d{2}):(?P<second>\d{2})'
    exp = df['Date/Time'].str.extract(pat,expand=True)
    exp['hour'] = exp['hour'].replace(24,0)
    exp['year'] = 2014

    df = df.set_index(pd.to_datetime(exp))

    return df


def to_C(F):
    C = (F-32) * (5/9)
    return (C)


def get_volume_needed(kW, peakLoad):
    usefulTemp = calc_t_cutout(peakLoad)
    deltaT = (storageTemperature) - (usefulTemp)    
    
    adders = (100-standbyLosses) # should bring volume needed up. 
    m = ((kW*3600) / (c_p * deltaT)) * (100/adders)
    
    return(m/1000)


def calc_t_cutout(kW):
    # this is all based on a rough rule to size flow rates to 1 GPM per 11,000 BTU. 
    # source: https://www.pexuniverse.com/how-size-circulator-pump can make better in the future. 
    # this does not paint the whole picture but is an estimate
    if (kW != 0):
        flowRate = 0.0705 * kW #[m^3/hr]
        t_cut = ((kW / (flowRate * 997 * 4.2)) * 3600/hydronicEfficiency) + roomTemperature

    else:
        t_cut = roomTemperature
        
    return (t_cut)
    
    # close enough... 
    
def get_gas_use(df, storageTime):

    timeShift = str(storageTime) + 'H'
    df2 = df.resample(timeShift).sum()[['Gas:Facility [kW](Hourly)']]
    val = df2.max()

    return (val[0])


def get_shift_percentage(df, maxLoad, storageTime):
    
    timeBlocks = (365*24) / storageTime
    timeShift = str(storageTime) + 'H'
    
    df2 = df.resample(timeShift).sum()[['Gas:Facility [kW](Hourly)']]
    
    addressedChunks = df2[df2['Gas:Facility [kW](Hourly)'] < maxLoad]
    percent = (len(addressedChunks)/timeBlocks) * 100
    return (percent)


def calc_expansion(volume):
    
    dT = (to_C(storageTemperature) - 23)
    return (volume * beta * dT)


In [47]:
# OLD STORAGE
highGasUse24 = create_df('HIGH.csv' , 24).sort_values(by='city')
lowGasUse24 = create_df('LOW.csv' , 24).sort_values(by='city')

highGasUse12 = create_df('HIGH.csv', 12).sort_values(by='city')
lowGasUse12 = create_df('LOW.csv', 12).sort_values(by='city')

highGasUse4 = create_df('HIGH.csv', 4).sort_values(by='city')
lowGasUse4 = create_df('LOW.csv', 4).sort_values(by='city')



### There are three types of thermal battery we can provide to a home: 


- Grid-Services 
    - Sized to shift energy use away from peak times, using TOU (Time-Of-Use) electricity pricing to save customer money and address duck curve. 
    - Depends on 'who the customer is'
    - COP ARBITRAGE also relevant here
    - 4-hour battery, larger heat pump
- Rooftop PV (Photovoltaic)
    - Sized to fully shift evening/night-time load, allowing for COP (Coefficient of Performance) arbitrage and rooftop PV to fully power home. 
    - TO DO: EXPLAIN FURTHER HOW COP ARBITRAGE WORKS
    - 12 hour battery, small heat pump
- Grid Independence 
    - Full thermal energy backup to keep house warm for several days beyond failure.
    - 24+ hour battery

The proper size for a particular home depends on:
1. Space available for Thermal Storage
2. Level of load shifting to be addressed
3. Load profile/energy envelope



Can compare cost savings to different things...
compare to: 
1. no thermal storage
2. cost of 


Can display what this amount of storage *could* look like for different homes. Roughly, the cost of storage for each of these will be different. more storage ultimately costs less per gallon. 

100-500 gallons $5 / gallon icorporated into the home

500-2000 gallons $2.5 / gallon attached shed

3000+ gallons $1 / gallon large drum



### Visualizing Storage
100-500 GALLONS
Can be stacked and fit into existing boiler room in retrofit buildings. 
![](./media/sHeat_withMan.png)

500-2000 GALLONS 
![](./media/attached_shed.png)

2000+ GALLONS

![](./media/drum_storage.png)

In [48]:
# Plot Storage Required By Location as Bar Graph

cities = highGasUse12['city'].to_list()

gal_low24 = (lowGasUse24['volume']).tolist()
gal_hi24 = (highGasUse24['volume']).tolist()

gal_low12 = (lowGasUse12['volume']).tolist()
gal_hi12 = (highGasUse12['volume']).tolist()

gal_low4 = (lowGasUse4['volume']).tolist()
gal_hi4 = (highGasUse4['volume']).tolist()
#kWhSize = newPANDA['heat_Need_LOW'].tolist()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=cities,
    y=gal_hi4,
    name='UTILITY (4HR SHIFT)',
#    marker_color='lightsalmon'
))


fig.add_trace(go.Bar(
    x=cities,
    y=gal_hi12,
    name='ROOFTOP SOLAR (12HR SHIFT)',
    marker_color='lightsalmon'
))


fig.add_trace(go.Bar(
    x=cities,
    y=gal_hi24,
    name='GRID INDEPENDENCE (24HR SHIFT)',
    marker_color='indianred'
))


cities = lowGasUse12['city'].to_list()



# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(yaxis_title ='gallons', title='VOLUME NEEDED TO SHIFT HIGH LOAD HOMES', barmode='group', xaxis_tickangle=-45)
fig.show()


fig = go.Figure()

fig.add_trace(go.Bar(
    x=cities,
    y=gal_low4,
    name='UTILITY (4HR SHIFT)',
#    marker_color='lightsalmon'
))


fig.add_trace(go.Bar(
    x=cities,
    y=gal_low12,
    name='ROOFTOP SOLAR (12HR SHIFT)',
    marker_color='lightsalmon'
))


fig.add_trace(go.Bar(
    x=cities,
    y=gal_low24,
    name='GRID INDEPENDENCE (24HR SHIFT)',
    marker_color='indianred'
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(yaxis_title ='gallons', title='VOLUME NEEDED TO SHIFT LOW LOAD HOMES', barmode='group', xaxis_tickangle=-45)
fig.show()

In [49]:
# Plot Storage Required By Location as Bar Graph

cities = highGasUse12['city'].to_list()

gal_low24 = (lowGasUse24['volume']).tolist()
gal_hi24 = (highGasUse24['volume']).tolist()

gal_low12 = (lowGasUse12['volume']).tolist()
gal_hi12 = (highGasUse12['volume']).tolist()

gal_low4 = (lowGasUse4['volume']).tolist()
gal_hi4 = (highGasUse4['volume']).tolist()
#kWhSize = newPANDA['heat_Need_LOW'].tolist()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=cities,
    y=gal_low24,
    name='24HR GRID INDEPENDENCE',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=cities,
    y=gal_hi24,
    name='HIGH LOAD storage [24hr shift]',
    marker_color='lightsalmon'
))

fig.add_trace(go.Bar(
    x=cities,
    y=gal_low12,
    name='LOW LOAD storage [12hr shift]',
#    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=cities,
    y=gal_hi12,
    name='HIGH LOAD storage [12hr shift]',
#    marker_color='lightsalmon'
))

fig.add_trace(go.Bar(
    x=cities,
    y=gal_low4,
    name='LOW LOAD storage [4hr shift]',
#    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=cities,
    y=gal_hi4,
    name='HIGH LOAD storage [4hr shift]',
#    marker_color='lightsalmon'
))



# fig.add_trace(go.Bar(
#     x=cities,
#     y=kWhSize,
#     name='Effective Battery Size',
#     marker_color='lightsalmon'
# ))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(yaxis_title ='gallons', title='VOLUME NEEDED TO SHIFT LOADS', barmode='group', xaxis_tickangle=-45)
fig.show()

### TODO COST SAVINGS

In [50]:
# dollars per kWh stored, Tesla powerwall
dpkwh = 7600 / 13.5 # using cost of powerwall, no installation**

lowGasUse4['storageCost'] = (lowGasUse4['volume'] * 5) / lowGasUse4['heatNeed']

lowGasUse4['new'] = lowGasUse4['volume'].apply(lambda x: 'True' if x <= 150 else 'False')

lowGasUse4


,tmy3,city,heatNeed,peakLoad,volume,shiftPercent,expansion,storageCost,new
5,702730,Anchorage,36.446578,10.208369,301.891731,42.191781,-0.295854,41.415648,False
6,726185,Augusta,38.945582,11.392054,322.591304,53.150685,-0.316139,41.415648,False
8,722280,Birmingham,16.083043,4.286480,133.217927,86.575342,-0.130554,41.415648,True
3,725090,Boston-Logan,32.873211,9.397890,272.293066,55.616438,-0.266847,41.415648,False
0,724699,Boulder-Broomfield-Jefferson,28.817160,8.119554,238.696272,50.958904,-0.233922,41.415648,False
9,726797,Bozeman-Gallatin,41.930040,12.359991,347.311955,53.698630,-0.340366,41.415648,False
2,724463,Kansas,18.548873,5.509993,153.642719,72.876712,-0.150570,41.415648,False
7,726410,Madison-Dane,42.089799,12.243027,348.635263,55.616438,-0.341663,41.415648,False
1,723085,Norfolk,13.596231,3.982023,112.619344,79.178082,-0.110367,41.415648,True
13,724930,Oakland,8.403026,2.807151,69.603356,3.013699,-0.068211,41.415648,True


### TODO: EXPLAIN COP ARBITRAGE BETTER


From a chart provided by SANDEN on their SANCO2 heat pumps, we can break out the COP with respect to ambient temperature. 

In [91]:
# realistic to use 150F temperature setting for initial data entry, will put them all in at some point in the future. 

T = [-13, -4, 5, 14, 23, 32, 41, 50, 59, 68, 77, 86, 95, 104]
cop_140F = [1.7, 2.0, 2.2, 2.5, 2.9, 3.2, 3.9, 4.7, 4.8, 5.2, 5.0, 4.6, 4.3, 4.0]
cop_150F = [1.7, 1.9, 2.2, 2.6, 3.0, 3.4, 3.9, 4.3, 4.5, 4.8, 4.6, 4.4, 4.2, 4.1]
cop_160F = [1.5, 1.9, 2.1, 2.5, 2.9, 3.3, 3.7, 4.0, 4.2, 4.3, 4.3, 4.2, 4.1, 4.0]

df = pd.DataFrame(list(zip(T, cop_140F, cop_150F, cop_160F)), 
                  columns = ['ambient_F', '140F', '150F', '160F'])

df['ambient_C'] = to_C(df['ambient_F'])


fig = go.Figure()

fig.add_trace(go.Scatter(x = T, y = cop_140F, name = '140F'))
fig.add_trace(go.Scatter(x = T, y = cop_150F, name = '150F'))
fig.add_trace(go.Scatter(x = T, y = cop_160F, name = '160F'))

fig.add_annotation(y=max(cop_140F), x = 68, text = 'COP 5.2')

fig.update_layout(title='HEAT PUMP COP w.r.t ambient temperature')

fig.show()

In [111]:
augusta = pd.read_csv('./data/726185TYA.csv', skiprows=[0])

augusta = augusta[(augusta['Date (MM/DD/YYYY)'] > '01/01/1997')]
augusta



,Date (MM/DD/YYYY),Time (HH:MM),ETR (W/m^2),ETRN (W/m^2),GHI (W/m^2),GHI source,GHI uncert (%),DNI (W/m^2),DNI source,DNI uncert (%),...,Alb (unitless),Alb source,Alb uncert (code),Lprecip depth (mm),Lprecip quantity (hr),Lprecip source,Lprecip uncert (code),PresWth (METAR code),PresWth source,PresWth uncert (code)
24,01/02/1994,01:00,0,0,0,1,0,0,1,0,...,0.15,F,8,0,1,D,9,0,C,8
25,01/02/1994,02:00,0,0,0,1,0,0,1,0,...,0.15,F,8,0,1,D,9,0,C,8
26,01/02/1994,03:00,0,0,0,1,0,0,1,0,...,0.15,F,8,0,1,D,9,0,C,8
27,01/02/1994,04:00,0,0,0,1,0,0,1,0,...,0.15,F,8,0,1,D,9,0,C,8
28,01/02/1994,05:00,0,0,0,1,0,0,1,0,...,0.15,F,8,0,1,D,9,0,C,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12/31/1997,20:00,0,0,0,1,0,0,1,0,...,0.16,F,8,0,1,D,9,87,C,8
8756,12/31/1997,21:00,0,0,0,1,0,0,1,0,...,0.16,F,8,0,1,D,9,87,C,8
8757,12/31/1997,22:00,0,0,0,1,0,0,1,0,...,0.16,F,8,0,1,D,9,87,C,8
8758,12/31/1997,23:00,0,0,0,1,0,0,1,0,...,0.16,F,8,0,1,D,9,87,C,8


In [19]:
df = px.data.gapminder().query("continent != 'Asia'") # remove Asia for visibility

fig = px.line(df, x = 'year', y = 'lifeExp', color = 'continent', line_group = 'country', hover_name = 'country')

fig.show()

df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
12,Albania,Europe,1952,55.230,1282697,1601.056136,ALB,8
13,Albania,Europe,1957,59.280,1476505,1942.284244,ALB,8
14,Albania,Europe,1962,64.820,1728137,2312.888958,ALB,8
15,Albania,Europe,1967,66.220,1984060,2760.196931,ALB,8
16,Albania,Europe,1972,67.690,2263554,3313.422188,ALB,8
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [12]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
12,Albania,Europe,1952,55.230,1282697,1601.056136,ALB,8
13,Albania,Europe,1957,59.280,1476505,1942.284244,ALB,8
14,Albania,Europe,1962,64.820,1728137,2312.888958,ALB,8
15,Albania,Europe,1967,66.220,1984060,2760.196931,ALB,8
16,Albania,Europe,1972,67.690,2263554,3313.422188,ALB,8
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


### Volume Needed:


### TODO: FIGURE OUT... ! (below)


### a little on $T_{cutoff}$

$$ T_t > \frac{H_l}{k_x} + T_{room} = T_{cutoff} $$


HOW THE FUCK DO I FIND $K_X$



Although it may be possible to throw a k_x value out with a chart like this.  
![](./media/design_load_variance.png)




based [on this](https://www.pexuniverse.com/how-size-circulator-pump) rule of thumb value for flow rate depending on load:

$1GPM = 11,000 \frac{BTU}{hr}$

93.76403156480481

0.7991638181818181
3.5205454545454544
33.25309957934367
33.19463862712758


### TODO: BASIC COMERCIAL ANALYSIS
